In [1]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =20

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

# MODEL CONFIG
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
os.chdir("data")

In [3]:
df = pd.read_csv("DatafinitiElectronicsProductsPricingData_Cleaned_dataset.csv")

In [4]:
dx = pd.read_csv("Products.csv") # this file contains product names

In [5]:
to_keep = dx["name"]
df = df[df['name'].isin(to_keep)]

In [6]:
df = df[(df["Year"]==2017) | (df["Year"]==2018)]

## Price Elasticity for all Products.

In [7]:
df["Year"].value_counts()

2017    5132
2018    1930
Name: Year, dtype: int64

In [8]:
#dfx = df[df["Category_name"].str.contains("speaker",case=False)]
#dfx= df[(df["Year"]==2017) | (df["Year"]==2018)]
dfx=df

In [9]:
dfx = dfx.reset_index(drop=True)

In [10]:
dfx.sort_values(by=['Formatted_date'],ignore_index=True)

,id,Actual_price,Discount_price,Product_availability,Condition,Currency,Country,isSale,merchant,brand,Description,Category_name,primaryCategories,name,weight_in_kg,Impression,Day,Month,Year,Week,Month_Name,Day_Name,Formatted_date,Disc_Per,Total_sale
0,AVpfWOIFLJeJML43...,137.99,137.99,Yes,New,USD,USA,False,Walmart.com,Alpine,"Electronics,Home...","car, gps, dash",Electronics,Alpine - UTE - A...,1.610252,1,7,1,2017,1,January,Saturday,2017-01-07,0.0,137.99
1,AVphibxI1cnluZ0-...,199.49,199.49,Yes,New,USD,USA,False,Walmart.com,Magellan,"Electronics,GPS ...","car, gps, dash",Electronics,Magellan Roadmat...,5.080230,1,8,1,2017,1,January,Sunday,2017-01-08,0.0,199.49
2,AVpflKqwLJeJML43...,4159.97,4159.97,Yes,Refurbished,USD,USA,False,Walmart.com,LG,"4K Ultra HD TVs,...","tv, 4k, television",Electronics,LG Electronics O...,31.887518,1,8,1,2017,1,January,Sunday,2017-01-08,0.0,4159.97
3,AVpflKqwLJeJML43...,4997.00,4997.00,Yes,New,USD,USA,False,Walmart.com,LG,"4K Ultra HD TVs,...","tv, 4k, television",Electronics,LG Electronics O...,31.887518,1,8,1,2017,1,January,Sunday,2017-01-08,0.0,4997.00
4,AVphAiVC1cnluZ0-...,96.43,96.43,Yes,New,USD,USA,False,Walmart.com,Razer,"Computers,Office...","headset, game, c...",Electronics,Razer Kraken Pro...,0.680388,1,14,1,2017,2,January,Saturday,2017-01-14,0.0,96.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7057,AV1YFFFzGV-KLJ3a...,59.00,59.00,Yes,New,USD,USA,False,bhphotovideo.com,Netgear,"Computers,Wi-Fi ...","networking, adap...",Electronics,ProSafe 16-Port ...,0.907184,2,25,7,2018,30,July,Wednesday,2018-07-25,0.0,118.00
7058,AVqVGZICnnc1JgDc...,499.99,499.99,Yes,New,USD,USA,False,bhphotovideo.com,Samsung,"Computers,Shop L...","laptop, computer",Electronics,12.3 32GB Multi-...,1.152124,1,25,7,2018,30,July,Wednesday,2018-07-25,0.0,499.99
7059,AVpfWurLLJeJML43...,99.90,99.90,Yes,New,USD,USA,False,bhphotovideo.com,NZXT,"Computers,Cases ...","case, computer",Electronics,Source 530 Full ...,9.162558,1,25,7,2018,30,July,Wednesday,2018-07-25,0.0,99.90
7060,AVpjV8a21cnluZ0-...,72.46,72.46,Yes,New,USD,USA,False,bhphotovideo.com,Fujifilm,"Instant Cameras,...","camera, shoot, p...",Electronics,instax mini 8 In...,4.898794,1,25,7,2018,30,July,Wednesday,2018-07-25,0.0,72.46


In [11]:
#Grouping data by name and Week number.
#dfx_mod = dfx.groupby(['name', "Month"]).agg({'Disc_Per':'mean','Impression':'sum',"Category_name":lambda x: pd.Series.mode(x)[0],"Formatted_date":lambda x: pd.Series.mode(x)[0]}).reset_index()
dfx_mod = dfx

In [12]:
#Rearranging
#dfx_mod = dfx_mod[["name","Category_name","Disc_Per","Impression","Month","Formatted_date"]]

In [13]:
dfx_mod['Date']= pd.to_datetime(dfx_mod['Formatted_date'])

In [14]:

dfx_mod.sort_values("Month",ignore_index=True,inplace=True)

In [15]:
# sub_df.info()
# converting object data type to int
dfx_mod['Condition'] = np.where(dfx_mod['Condition'] == 'new',1,
                      np.where(dfx_mod['Condition'] == 'refurbished',2,
                      np.where(dfx_mod['Condition'] == 'used',0,0)))


dfx_mod['Product_availability'] = np.where(dfx_mod['Product_availability'] == 'yes',1,
                                 np.where(dfx_mod['Product_availability'] == 'no',0,
                                 np.where(dfx_mod['Product_availability'] == 'other',2,2)))


dfx_mod['Currency']= np.where(dfx_mod['Currency'] == 'USD','1',
                np.where(dfx_mod['Currency'] == 'CAD',"2",
                np.where(dfx_mod['Currency'] == 'SGD',"3",
                np.where(dfx_mod['Currency'] == 'EUR',"4",
                np.where(dfx_mod['Currency'] == 'GBP',"5","6"
                )))))

dfx_mod['isSale'] = np.where(dfx_mod['isSale'] == True,1,
                      np.where(dfx_mod['isSale'] == False,0,0))

In [16]:
dfx_mod['name'].nunique()

424

In [17]:
# we will drop this column
# df['name'].value_counts()
dfx_mod['namex'], name_categories = pd.factorize(dfx_mod['name'])
mapping_df = pd.DataFrame({'name': name_categories, 'namex': range(len(name_categories))})
print(mapping_df)
dfx_mod['name'] = dfx_mod['namex']
dfx_mod.drop(columns='namex',inplace=True)

                    name  namex
0    Elite SC-LX501 7...      0
1    Power Acoustik E...      1
2    PYLE - Amplifier...      2
3    Details About So...      3
4    Sennheiser - RS ...      4
..                   ...    ...
419  J5205 32-Class F...    419
420  Sony XBR55X700D ...    420
421  Acer Predator G1...    421
422  SiriusXM SXEZR1V...    422
423  Monster SuperSta...    423

[424 rows x 2 columns]


In [18]:
mapping_df.to_csv("prod_name.csv",index=True)

In [19]:
dfx_mod.columns

Index(['id', 'Actual_price', 'Discount_price', 'Product_availability',
       'Condition', 'Currency', 'Country', 'isSale', 'merchant', 'brand',
       'Description', 'Category_name', 'primaryCategories', 'name',
       'weight_in_kg', 'Impression', 'Day', 'Month', 'Year', 'Week',
       'Month_Name', 'Day_Name', 'Formatted_date', 'Disc_Per', 'Total_sale',
       'Date'],
      dtype='object')

In [20]:
#dfx_mod = dfx_mod[["name","Actual_price","Product_availability","Condition","isSale","Currency","Month","Discount_price"]]
dfx_mod = dfx_mod[["name","Actual_price","Product_availability","Condition","isSale","Month","Impression","Discount_price"]]

for col in dfx_mod.keys():
    dfx_mod[col] = dfx_mod[col].astype(int)

dfx_mod.sort_values("name",inplace=True)




### FINAL DATA FRAME

In [21]:
dfx_mod

,name,Actual_price,Product_availability,Condition,isSale,Month,Impression,Discount_price
0,0,498,2,0,0,1,2,498
4376,0,718,2,0,0,8,1,718
4379,0,900,2,0,0,8,1,900
4386,0,738,2,0,0,8,2,738
4395,0,719,2,0,0,8,1,719
...,...,...,...,...,...,...,...,...
5850,423,118,2,0,1,10,1,108
5851,423,119,2,0,0,10,2,119
6806,423,118,2,0,1,12,1,90
4918,423,118,2,0,0,9,2,118


#### X-Values = PRICE, Actua_price, Product_availablity, condition, isSale,product_name, week_no

In [22]:
# X data
x_val = dfx_mod.drop('Discount_price', axis=1)

#### Y-Values = Discount Price

In [23]:
# Y data
y_val = dfx_mod['Discount_price']

### Train,Test Splitter

In [24]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.2, random_state=42)


### Linera Model

In [25]:
# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [26]:
ls = x_val.columns
coeff = pd.DataFrame(model.coef_, columns={"coeff"})
coeff["name"]= ls
coeff


,coeff,name
0,-2.298975e-02,name
1,8.869765e-01,Actual_price
2,-3.286260e-14,Product_availabi...
3,7.105427e-15,Condition
4,-2.389817e+02,isSale
5,-1.354994e+00,Month
6,6.268771e-01,Impression


### Evaluation

In [27]:
def evaluater(eval_type,mod,xval,yval):
    y_pred = mod.predict(xval)

    print(eval_type.center(70,"*"))
    mae = mean_absolute_error(yval, y_pred)
    mse = mean_squared_error(yval, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(yval, y_pred)

    # Print the evaluation metrics
    print('Mean absolute error (MAE): {:.2f}'.format(mae))
    print('Mean squared error (MSE): {:.2f}'.format(mse))
    print('Root mean squared error (RMSE): {:.2f}'.format(mae))
    print('R-squared (R2) score: {:.2f}'.format(r2)) 
    print("*"*70)

In [28]:
evaluater("Train data evaluation",model,X_train,y_train)

************************Train data evaluation*************************
Mean absolute error (MAE): 82.66
Mean squared error (MSE): 30206.62
Root mean squared error (RMSE): 82.66
R-squared (R2) score: 0.96
**********************************************************************


In [29]:
evaluater("Test data evaluation",model,X_test,y_test)

*************************Test data evaluation*************************
Mean absolute error (MAE): 87.76
Mean squared error (MSE): 32087.39
Root mean squared error (RMSE): 87.76
R-squared (R2) score: 0.95
**********************************************************************


In [51]:
#Combining data from 03.Model-01 PED data with product name encoding data.
d2 = pd.read_csv("PED-data-raw.csv") #from 03.Model-01
d3 = pd.read_csv("ProdName2Num.csv") #from 03.Model-02 -name encoding
merged_df = pd.merge(d3, d2, on='name', how='left')
merged_df = merged_df.drop(merged_df.columns[0], axis=1)
merged_df.to_csv("PED-data-final.csv",index=False)

dv = pd.read_csv("PED-data-final.csv")
merged_df

,name,namex,category,price_elasticity,price_mean,impressions_mean,imp_mean_pct,intercept,slope
0,Elite SC-LX501 7...,0,"receiver, amplif...",-0.141573,783.644740,37.263158,0.311540,0.355646,-0.016351
1,Power Acoustik E...,1,"car, speaker, su...",-0.133556,159.897316,22.263158,0.250870,0.284375,-0.018890
2,PYLE - Amplifier...,2,"receiver, amplif...",-3.762006,194.986272,6.684211,0.037789,0.179952,-0.019573
3,Details About So...,3,"tv, television, led",-0.275314,1597.215140,7.105263,0.355582,0.453479,-0.031113
4,Sennheiser - RS ...,4,headphone,-0.257778,259.347789,13.526316,0.773062,0.972341,-0.090150
...,...,...,...,...,...,...,...,...,...
419,J5205 32-Class F...,419,"tv, television, led",1.156465,295.697119,64.842105,-0.041168,0.006441,-0.011486
420,Sony XBR55X700D ...,420,"tv, 4k, television",-0.580283,781.410234,97.789474,0.105466,0.166667,-0.058140
421,Acer Predator G1...,421,desktop,-0.431302,1727.568947,9.473684,-0.089647,-0.128312,0.004833
422,SiriusXM SXEZR1V...,422,"radio, portable,...",-0.727273,70.603333,3.000000,0.036683,0.063361,-0.003727


In [73]:
def preditor(a,b,c,d,e,f,g):
    global gname,gcat,gpe,gpm,gim,gint,gslope
    y_pred = model.predict([[a,b,c,d,e,f,g]])
    loader(a)
    abs_x = np.abs(np.float(y_pred))
    percentage_diff = ((abs_x - b) / b) * 100 #abs
    print(f"Product Name: {gname} ")
    print(f"Product Catergory: {gcat} ")
    print(f"Price Elasticity: {gpe} ")
    #print(f"Price Eldwasticity: {gslope} ") ##
    print(f"Discount Percentage to Set: {round(percentage_diff,2)} %")
    new_p = np.abs(((b/100)*percentage_diff)-b) #np.abs
    #new_p = b * (1 + gpe)
    #new_p = b / (1 + ((gim*h) / gpe))
    #new_p = (gim ) * (b / h)
    print(f"New Prediction Price: {np.abs(round(new_p,2))}")

def loader(x):
        global gname,gcat,gpe,gpm,gim,gint,gslope
        g1 = dv[dv['namex'] == x]
        #print(g1)
        gname = g1["name"].iloc[0]
        gcat=g1["category"].iloc[0]
        gpe=g1["price_elasticity"].iloc[0]
        gpm=g1["price_mean"].iloc[0]
        gim=g1["impressions_mean"].iloc[0]
        gint=g1["intercept"].iloc[0]
        gslope=g1["slope"].iloc[0]

In [74]:
a= 15   #Product ID
b= 850  #Actual Price
c= 2    #Condition
d= 0    #Availability
e= 0    #on Sale 
f= 1    #Month 
h= 22   #Impressions

preditor(a,b,c,d,e,f,h)

Product Name: 4TB My Book Desktop USB 3.0 External Hard Drive 
Product Catergory: drive, storage, hard 
Price Elasticity: -2.1666666666666674 
Discount Percentage to Set: -1.79 %
New Prediction Price: 865.18
